# Zeroth Pipeline

This pipeline compares zeroth order GD against standard GD. Three different zeroth order approximations --- one difference, two difference, and coordinate --- are tested.

### Imports

In [ ]:
import os
import sys

p = os.path.abspath('../src/')
if p not in sys.path:
    sys.path.append(p)

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import tqdm

from difference_methods import one_point_estimate, two_point_estimate, coordinate_estimate
from linear_zeroth_function import Linear_Zeroth
from sigmoid_zeroth_function import Sigmoid_Zeroth
from relu_zeroth_function import ReLU_Zeroth

### Data preprocessing

In [ ]:
# Load training data
data_train = pd.read_csv('../data/pendigits.csv')

print(data_train)

In [ ]:
# extract frequency of each class
class_freq = data_train['class'].value_counts()
class_freq = list(class_freq.sort_index())

nb_classes = len(class_freq)
nb_features = data_train.shape[1]-1

# Visual data
graph = plt.bar(list(range(nb_classes)), class_freq)
plt.xticks(list(range(nb_classes)))
plt.ylabel('Frequency')
plt.xlabel('Class')
plt.title('Full data')

plt.show() # balanced data set

In [ ]:
# Preprocess data for training

# Convert pandas dataframe to array
data_train_array = data_train.values

# Split x (features) and y (targets)
x_array = data_train_array[:, :nb_features]
y_array = data_train_array[:, nb_features]

# Tensorify
X = torch.tensor(x_array, dtype=torch.float)
Y = torch.tensor(y_array, dtype=torch.long)

print(X.shape)

### Network Setup

In [ ]:
# INPUT/OUTPUT layer sizes
INPUT_NEURONS = nb_features
OUTPUT_NEURONS = nb_classes

In [ ]:
# Hyperparameters
hidden_neurons_range = list(range(min(INPUT_NEURONS, OUTPUT_NEURONS), max(INPUT_NEURONS, OUTPUT_NEURONS)+1, 2))
mu_range = [1e-3, 1e-2, 1e-1]
n_range = list(range(1, min(INPUT_NEURONS, OUTPUT_NEURONS), 4))
difference_method_range = ['one','two','coord','sgd']

# Display hyperparameters:
print(f"Hidden Neuron Range: {hidden_neurons_range}")
print(f"Mu Range: {mu_range}")
print(f"N Range: {n_range}")
print(f"Descent Methods: {difference_method_range}")

In [ ]:
# Network architectures: 1 hidden layer

# Activation functions
# Normal network with sigmoid
def create_standard_sigmoid_network(hidden_neurons, mu, n, difference_method):
    torch.manual_seed(1) # so that same network init with same weights at each creation
    return torch.nn.Sequential(
        torch.nn.Linear(INPUT_NEURONS, hidden_neurons),
        torch.nn.Sigmoid(),
        torch.nn.Linear(hidden_neurons, OUTPUT_NEURONS)
    )

# Full Zeroth Order sigmoid
def create_zeroth_sigmoid_network(hidden_neurons, mu, n, difference_method):
    torch.manual_seed(1) # so that same network init with same weights at each creation
    return torch.nn.Sequential(
        Linear_Zeroth(INPUT_NEURONS, hidden_neurons, bias=False, difference_method=difference_method, mu=mu, n=n),
        Sigmoid_Zeroth(difference_method=difference_method, mu=mu, n=n),
        Linear_Zeroth(hidden_neurons, OUTPUT_NEURONS, bias=False, difference_method=difference_method, mu=mu, n=n)
)

# Normal network with relu
def create_standard_relu_network(hidden_neurons, mu, n, difference_method):
    torch.manual_seed(1) # so that same network init with same weights at each creation
    return torch.nn.Sequential(
        torch.nn.Linear(INPUT_NEURONS, hidden_neurons),
        torch.nn.ReLU(),
        torch.nn.Linear(hidden_neurons, OUTPUT_NEURONS)
    )

# Full Zeroth Order relu
def create_zeroth_relu_network(hidden_neurons, mu, n, difference_method):
    torch.manual_seed(1) # so that same network init with same weights at each creation
    return torch.nn.Sequential(
        Linear_Zeroth(INPUT_NEURONS, hidden_neurons, bias=False, difference_method=difference_method, mu=mu, n=n),
        ReLU_Zeroth(difference_method=difference_method, mu=mu, n=n),
        Linear_Zeroth(hidden_neurons, OUTPUT_NEURONS, bias=False, difference_method=difference_method, mu=mu, n=n),
)

std_network_funs = [
    ("standard_sigmoid", create_standard_sigmoid_network),
    ("standard_relu", create_standard_relu_network),
]

zero_network_funs = [
    ("zeroth_sigmoid", create_zeroth_sigmoid_network),
    ("zeroth_relu", create_zeroth_relu_network),
]


### Train and Plot Losses

In [ ]:
# Training hyperparameters
# Training constants
LR = 0.01
NUM_EPOCH = 500

# Loss function
loss_func = torch.nn.CrossEntropyLoss()

In [ ]:
# Plot or save? False => Save
plot = False
path = "plots/"

In [ ]:
# Train each network and plot the loss:
for net_idx, network in enumerate(zero_network_funs):
    name, network_fun = network
    for hidden_neurons in hidden_neurons_range:
        for mu in mu_range:
            for n in n_range:
                plt.figure()
                for diff_method in difference_method_range:
                    print(f"Network: {name}, Hidden neurons: {hidden_neurons}, Mu: {mu}, N: {n} \n=========")

                    # Create new instance of network
                    if diff_method == "sgd":
                        network_fun = std_network_funs[net_idx][1]
                    network = network_fun(hidden_neurons, mu, n, diff_method)

                    # Optimiser
                    optimiser = torch.optim.SGD(network.parameters(), lr=LR)

                    # store all losses for visualisation
                    all_losses = []

                    # train a neural network
                    for epoch in tqdm.tqdm(range(NUM_EPOCH)):
                        # Perform forward pass
                        Y_pred = network(X)
                        # Compute loss
                        loss = loss_func(Y_pred, Y)
                        all_losses.append(loss.item())

                        # print progress
                        if epoch % 50 == 0:
                            # convert three-column predicted Y values to one column for comparison
                            _, predicted = torch.max(torch.nn.functional.softmax(Y_pred,1), 1)
                            # calculate and print accuracy
                            total = predicted.size(0)
                            correct = predicted.data.numpy() == Y.data.numpy()
                            # Print
                            # print('Epoch [%d / %d] Loss: %.4f  Accuracy: %.2f %%' % (epoch + 1, NUM_EPOCH, loss.item(), 100 * sum(correct)/total))

                        # Clear the gradients before running the backward pass.
                        network.zero_grad()
                        # Perform backward pass
                        loss.backward()
                        # Step optimiser
                        optimiser.step()

                    # Plot
                    plt.plot(all_losses)
                    
                plt.xlabel("Epoch")
                plt.ylabel("Loss")
                plt.legend(["One", "Two", "Coord", "SGD"])
                if plot:
                    plt.show()
                else:
                    plt.savefig(f"{path}{name}_hn{hidden_neurons}_mu{mu}_n{n}.png")
                    plt.close()